# Library Imports

In [1]:
import itertools
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam, SGD
from keras.losses import BinaryCrossentropy
from keras.regularizers import l2, l1, l1_l2
from sklearn.metrics import accuracy_score, balanced_accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

In [2]:
class Colours:
    HEADER = '\033[95m'     # Parameters
    BLUE = '\033[94m'       # Class Distribution
    GREEN = '\033[92m'      # Accuracies
    RED = '\033[91m'        # Loss
    ENDC = '\033[0m'        # End Colours
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# Dataset Loading, Cleaning, & Splitting

In [3]:
# Load the dataset
data = pd.read_csv('diabetes.csv')

In [4]:
# Clean the data

# Glucose: Replace '0' values with NaN
data['Glucose'] = data['Glucose'].apply(lambda x: np.nan if x == 0 else x)

# BloodPressure: Replace values below 40 mm Hg with NaN
data['BloodPressure'] = data['BloodPressure'].apply(lambda x: np.nan if x < 40 else x)

# SkinThickness: Replace values below 10 mm with NaN
data['SkinThickness'] = data['SkinThickness'].apply(lambda x: np.nan if x < 10 else x)

# Insulin: Replace values above 400 µU/mL with NaN
data['Insulin'] = data['Insulin'].apply(lambda x: np.nan if x > 400 else x)

# BMI: Replace '0' values with NaN
data['BMI'] = data['BMI'].apply(lambda x: np.nan if x == 0 else x)

In [5]:
# Split the data into training, testing, and validation datasets

# Split the data into features and target
X = data.drop('Outcome', axis=1)
y = data['Outcome']

# Split the dataset into train, validation, and test sets
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.2, random_state=42, stratify=y_trainval)

In [ ]:
print(f"Class distribution: {Colours.BLUE}{np.bincount(y)}{Colours.ENDC}")

# Data Preprocessing

In [7]:
# Preprocess the data by performing KNN Imputation and scaling the dataset with RobustScaler

imputer = KNNImputer(n_neighbors=5)
scaler = RobustScaler()

# Impute and scale training dataset
X_train_imputed = imputer.fit_transform(X_train)
X_train_scaled = scaler.fit_transform(X_train_imputed)

# Impute and scale validation dataset
X_val_imputed = imputer.transform(X_val)
X_val_scaled = scaler.transform(X_val_imputed)

# Impute and scale trainval dataset
X_trainval_imputed = imputer.fit_transform(X_trainval)
X_trainval_scaled = scaler.fit_transform(X_trainval_imputed)

# Impute and scale testing dataset
X_test_imputed = imputer.transform(X_test)
X_test_scaled = scaler.transform(X_test_imputed)

# Dummy Classifier Pipeline

In [8]:
# Defining a dummy classifier pipeline
dummyPipeline = Pipeline([
    ('imputer', KNNImputer(n_neighbors=5)),
    ('scaler', RobustScaler()),
    ('classifier', DummyClassifier(strategy='stratified'))
])

# Standard Single Layer Perceptron

In [9]:
# Defining a standard single-layer perceptron
def ssl_perceptron(learning_rate=0.01):
    model = Sequential()

    model.add(Dense(1, input_dim=X_train.shape[1], activation='linear'))

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss=BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
    
    return model

# Optimized Single Layer Perceptron

In [10]:
# Defining the optimized single-layer perceptron
def osl_perceptron(learning_rate=0.01, activation='binary', regularization=None, lambda_=0.01, optimizer='adam'):
    model = Sequential()

    if regularization == 'l1':
        kernel_regularizer = l1(lambda_)
    elif regularization == 'l2':
        kernel_regularizer = l2(lambda_)
    elif regularization == 'elasticnet':
        kernel_regularizer = l1_l2(l1=lambda_, l2=lambda_)
    else:
        kernel_regularizer = None
    
    if activation == 'binary':
        model.add(Dense(1, input_dim=X_train.shape[1], activation='linear', kernel_regularizer=kernel_regularizer))
    else:
        model.add(Dense(1, input_dim=X_train.shape[1], activation=activation, kernel_regularizer=kernel_regularizer))

    if optimizer == 'adam':
        optimizer_instance = Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        optimizer_instance = SGD(learning_rate=learning_rate)

    model.compile(optimizer=optimizer_instance,
                  loss=BinaryCrossentropy(from_logits=True) if activation == 'binary' else BinaryCrossentropy(),
                  metrics=['accuracy'])
    
    return model

# Priliminary Model Training, Hyperparameter Tuning, & Validation Evaluation

In [ ]:
# Conduct hyperparameter tuning on the Optimized Single Layer Perceptron (OSLP) model

# Hyperparameter space
learning_rates = [0.001, 0.01]                      # Learning rate values to test
epochs_list = [500, 750]                            # Number of iterations (epochs) to test
activations = ['sigmoid', 'tanh', 'relu']           # Activation functions to test
regularizations = [None, 'l1', 'l2', 'elasticnet']  # Regularization types to test
lambda_values = [0.001, 0.01]                       # Regularization strengths to test
optimizers = ['adam', 'sgd']                        # Optimization methods to test

# Tuning metrics
best_accuracy = 0
best_params = {}

# Hyperparameter tuning loop
for learning_rate, epochs, activation, regularization, lambda_, optimizer in itertools.product(learning_rates, epochs_list, activations, regularizations, lambda_values, optimizers):
    print(f"Training with learning_rate={learning_rate}, epochs={epochs}, activation={activation}, regularization={regularization}, lambda_={lambda_}, optimizer={optimizer}")

    # Build and train the model
    model = osl_perceptron(learning_rate=learning_rate, activation=activation, optimizer=optimizer, regularization=regularization, lambda_=lambda_)
    model.fit(X_train_scaled, y_train, epochs=epochs, batch_size=32, verbose=0, validation_data=(X_val_scaled, y_val))

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val_scaled, y_val, verbose=0)
    print(f"Validation Accuracy: {Colours.GREEN}{val_accuracy:.4f}{Colours.ENDC}, Validation Loss: {Colours.RED}{val_loss:.4f}{Colours.ENDC}")

    # Update best parameters if current model is better
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_params = {
            'learning_rate': learning_rate,
            'epochs': epochs,
            'activation': activation,
            'regularization': regularization,
            ''
            'optimizer': optimizer
        }

# Display the best parameters and accuracy
print(f"Best Parameters: {Colours.HEADER}{best_params}{Colours.ENDC}")
print(f"Best Validation Accuracy: {Colours.GREEN}{best_accuracy:.4f}{Colours.ENDC}")

In [ ]:
# Manually train the OSLP after determining ideal parameters

# Map best parameters to Perceptron parameters, instantiate, and train the Perceptron
best_oslp = osl_perceptron(learning_rate=best_params['learning_rate'],
                            activation=best_params['activation'],
                            regularization=best_params['regularization'],
                            lambda_=best_params['lambda_'],
                            optimizer=best_params['optimizer'])

train_record_oslp = best_oslp.fit(X_train_scaled, y_train, epochs=best_params['epochs'], batch_size=32, validation_data=(X_val_scaled, y_val), verbose=3)

In [ ]:
# Evaluate the OSLP model on the Validation dataset

# Use the trained perceptron to predict on the validation set
y_val_pred_oslp = (best_oslp.predict(X_val_scaled) > (0.0 if best_params['activation'] == 'binary' else 0.5)).astype(int).flatten()

# Calculate the accuracy
val_accuracy_oslp = accuracy_score(y_val, y_val_pred_oslp)
print(f"Validation Accuracy: {Colours.GREEN}{val_accuracy_oslp:.2f}{Colours.ENDC}")

# Calculate the balanced accuracy
val_balanced_accuracy_oslp = balanced_accuracy_score(y_val, y_val_pred_oslp)
print(f"Validation Balanced Accuracy: {Colours.GREEN}{val_balanced_accuracy_oslp:.2f}{Colours.ENDC}")

# Display a classification report
print(f"{Colours.BOLD}Validation Classification Report:{Colours.ENDC}")
print(classification_report(y_val, y_val_pred_oslp, target_names=['Non-Diabetic', 'Diabetic']))

In [ ]:
# Simple Single-Layer Perceptron (SSLP) Training
sslp = ssl_perceptron(learning_rate=best_params['learning_rate'])

train_record_sslp = sslp.fit(X_train_scaled, y_train, epochs=best_params['epochs'], batch_size=32, validation_data=(X_val_scaled, y_val), verbose=3)

In [ ]:
# Evaluate SSLP

# Use the trained perceptron to predict on the validation set
y_val_pred_sslp = (sslp.predict(X_val_scaled) > 0.0).astype(int).flatten()

# Calculate the accuracy
val_accuracy_sslp = accuracy_score(y_val, y_val_pred_sslp)
print(f"Validation Accuracy: {Colours.GREEN}{val_accuracy_sslp:.2f}{Colours.ENDC}")

# Calculate the balanced accuracy
val_balanced_accuracy_sslp = balanced_accuracy_score(y_val, y_val_pred_sslp)
print(f"Validation Balanced Accuracy: {Colours.GREEN}{val_balanced_accuracy_sslp:.2f}{Colours.ENDC}")

# Display a classification report
print(f"{Colours.BOLD}Validation Classification Report:{Colours.ENDC}")
print(classification_report(y_val, y_val_pred_sslp, target_names=['Non-Diabetic', 'Diabetic']))

In [ ]:
# Dummy Classifier

# Train
dummyPipeline.fit(X_train, y_train)

# Predict
dummy_val_pred = dummyPipeline.predict(X_val)

# Evaluate
print(f"{Colours.BOLD}Dummy Validation Classification Report:{Colours.ENDC}")
print(classification_report(y_val, dummy_val_pred, target_names=['Non-Diabetic', 'Diabetic']))

# Preliminary Training Result Visualizations

In [ ]:
# Plot of Training vs Validation Accuracy over Epochs (OSLP)
plt.figure(figsize=(12, 6))
plt.plot(train_record_oslp.history['accuracy'], label='Training Accuracy')
plt.plot(train_record_oslp.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy (OSLP)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot of Training vs Validation Loss over Epochs (OSLP)
plt.figure(figsize=(12, 6))
plt.plot(train_record_oslp.history['loss'], label='Training Loss')
plt.plot(train_record_oslp.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss (OSLP)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# True vs Predicted Labels (OSLP)
ConfusionMatrixDisplay.from_predictions(y_val, y_val_pred_oslp, display_labels=['Non-Diabetic', 'Diabetic'])
plt.show()

In [ ]:
# Plot of Training vs Validation Accuracy over Epochs (SSLP)
plt.figure(figsize=(12, 6))
plt.plot(train_record_sslp.history['accuracy'], label='Training Accuracy')
plt.plot(train_record_sslp.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training vs Validation Accuracy (SSLP)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot of Training vs Validation Loss over Epochs (SSLP)
plt.figure(figsize=(12, 6))
plt.plot(train_record_sslp.history['loss'], label='Training Loss')
plt.plot(train_record_sslp.history['val_loss'], label='Validation Loss')
plt.title('Training vs Validation Loss (SSLP)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# True vs Predicted Labels (SSLP)
ConfusionMatrixDisplay.from_predictions(y_val, y_val_pred_sslp, display_labels=['Non-Diabetic', 'Diabetic'])
plt.show()

# Secondary Model Training, Testing, & Evaluation

In [ ]:
# Re-train the OSLP with the combined trainval dataset and ideal parameters

# Map best parameters to Perceptron parameters, instantiate, and train the Perceptron
final_oslp = osl_perceptron(learning_rate=best_params['learning_rate'],
                            activation=best_params['activation'],
                            regularization=best_params['regularization'],
                            lambda_=best_params['lambda_'],
                            optimizer=best_params['optimizer'])

trainval_record_oslp = final_oslp.fit(X_trainval_scaled, y_trainval, epochs=best_params['epochs'], batch_size=32, validation_data=(X_test_scaled, y_test), verbose=3)

In [ ]:
# Evaluate the OSLP model on the Testing dataset

# Use the re-trained perceptron to predict on the testing set
y_test_pred_oslp = (final_oslp.predict(X_test_scaled) > (0.0 if best_params['activation'] == 'binary' else 0.5)).astype(int).flatten()

# Calculate the accuracy
test_accuracy = accuracy_score(y_test, y_test_pred_oslp)
print(f"Testing Accuracy: {Colours.GREEN}{test_accuracy:.2f}{Colours.ENDC}")

# Calculate the balanced accuracy
test_balanced_accuracy = balanced_accuracy_score(y_test, y_test_pred_oslp)
print(f"Testing Balanced Accuracy: {Colours.GREEN}{test_balanced_accuracy:.2f}{Colours.ENDC}")

print(f"{Colours.BOLD}Testing Classification Report:{Colours.ENDC}")
print(classification_report(y_test, y_test_pred_oslp, target_names=['Non-Diabetic', 'Diabetic']))

In [ ]:
# Simple Single-Layer Perceptron (SSLP) Re-Training
final_sslp = ssl_perceptron(learning_rate=best_params['learning_rate'])

trainval_record_sslp = final_sslp.fit(X_trainval_scaled, y_trainval, epochs=best_params['epochs'], batch_size=32, validation_data=(X_test_scaled, y_test), verbose=3)

In [ ]:
# Re-Evaluate SSLP

# Use the re-trained perceptron to predict on the testing set
y_test_pred_sslp = (final_sslp.predict(X_test_scaled) > 0.0).astype(int).flatten()

# Calculate the accuracy
test_accuracy_sslp = accuracy_score(y_test, y_test_pred_sslp)
print(f"Testing Accuracy: {Colours.GREEN}{test_accuracy_sslp:.2f}{Colours.ENDC}")

# Calculate the balanced accuracy
test_balanced_accuracy_sslp = balanced_accuracy_score(y_test, y_test_pred_sslp)
print(f"Testing Balanced Accuracy: {Colours.GREEN}{test_balanced_accuracy_sslp:.2f}{Colours.ENDC}")

# Display a classification report
print(f"{Colours.BOLD}Testing Classification Report:{Colours.ENDC}")
print(classification_report(y_test, y_test_pred_sslp, target_names=['Non-Diabetic', 'Diabetic']))

In [ ]:
# Dummy Classifier

# Train
dummyPipeline.fit(X_trainval, y_trainval)

# Predict
dummy_test_pred = dummyPipeline.predict(X_test)

# Evaluate
print(f"{Colours.BOLD}Dummy Testing Classification Report:{Colours.ENDC}")
print(classification_report(y_test, dummy_test_pred, target_names=['Non-Diabetic', 'Diabetic']))

# Secondary Training Result Visualizations

In [ ]:
# Plot of Training vs Testing Accuracy over Epochs (OSLP)
plt.figure(figsize=(12, 6))
plt.plot(trainval_record_oslp.history['accuracy'], label='Training Accuracy')
plt.plot(trainval_record_oslp.history['val_accuracy'], label='Testing Accuracy')
plt.title('Training vs Testing Accuracy (OSLP)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot of Training vs Testing Loss over Epochs (OSLP)
plt.figure(figsize=(12, 6))
plt.plot(trainval_record_oslp.history['loss'], label='Training Loss')
plt.plot(trainval_record_oslp.history['val_loss'], label='Testing Loss')
plt.title('Training vs Testing Loss (OSLP)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# True vs Predicted Labels (OSLP)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred_oslp, display_labels=['Non-Diabetic', 'Diabetic'])
plt.show()

In [ ]:
# Plot of Training vs Testing Accuracy over Epochs (SSLP)
plt.figure(figsize=(12, 6))
plt.plot(trainval_record_sslp.history['accuracy'], label='Training Accuracy')
plt.plot(trainval_record_sslp.history['val_accuracy'], label='Testing Accuracy')
plt.title('Training vs Testing Accuracy (SSLP)')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Plot of Training vs Testing Loss over Epochs (SSLP)
plt.figure(figsize=(12, 6))
plt.plot(trainval_record_sslp.history['loss'], label='Training Loss')
plt.plot(trainval_record_sslp.history['val_loss'], label='Testing Loss')
plt.title('Training vs Testing Loss (SSLP)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# True vs Predicted Labels (SSLP)
ConfusionMatrixDisplay.from_predictions(y_test, y_test_pred_sslp, display_labels=['Non-Diabetic', 'Diabetic'])
plt.show()